In [14]:
## The following packages were recommended based on the article found at https://towardsdatascience.com/clustering-geospatial-data-f0584f0b04ec
## for data
import numpy as np
import pandas as pd
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for geospatial
import folium
import geopy
## for machine learning
from sklearn import preprocessing, cluster
import scipy
## for deep learning
import minisom

## for binary and os operations
import glob
import os

In [11]:
all_csvs = glob.glob(os.path.join('.', '*_us.csv'))
epa_tri = pd.concat((pd.read_csv(f) for f in all_csvs), ignore_index=True)

/tmp/ipykernel_552/2961636835.py:2: DtypeWarning: Columns (15,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in all_csvs), ignore_index=True)
/tmp/ipykernel_552/2961636835.py:2: DtypeWarning: Columns (8,10,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in all_csvs), ignore_index=True)
/tmp/ipykernel_552/2961636835.py:2: DtypeWarning: Columns (15,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in all_csvs), ignore_index=True)
/tmp/ipykernel_552/2961636835.py:2: DtypeWarning: Columns (23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in all_csvs), ignore_index=True)
/tmp/ipykernel_552/2961636835.py:2: DtypeWarning: Columns (8,21,23,25) have mixed types. Specify dtype option on import or set low_memo

In [16]:
fda_cdi = pd.read_csv('CDI_with_pop_centers.csv')
fda_cdi.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3,Population,Latitude,Longitude,Centroid
0,2015,2015,MS,Mississippi,NVSS,Alcohol,Chronic liver disease mortality,NaN,NaN,Number,...,OVERALL,OVR,NaN,NaN,NaN,NaN,2844658,32.56642,-89.593164,"{""type"":""Point"", ""coordinates"":[-89.593164,32...."
1,2014,2014,MS,Mississippi,NVSS,Alcohol,Chronic liver disease mortality,NaN,"cases per 100,000",Age-adjusted Rate,...,OVERALL,OVR,NaN,NaN,NaN,NaN,2844658,32.56642,-89.593164,"{""type"":""Point"", ""coordinates"":[-89.593164,32...."
2,2016,2016,MS,Mississippi,NVSS,Alcohol,Chronic liver disease mortality,NaN,"cases per 100,000",Crude Rate,...,OVERALL,OVR,NaN,NaN,NaN,NaN,2844658,32.56642,-89.593164,"{""type"":""Point"", ""coordinates"":[-89.593164,32...."
3,2014,2014,MS,Mississippi,NVSS,Alcohol,Chronic liver disease mortality,NaN,"cases per 100,000",Crude Rate,...,OVERALL,OVR,NaN,NaN,NaN,NaN,2844658,32.56642,-89.593164,"{""type"":""Point"", ""coordinates"":[-89.593164,32...."
4,2017,2017,MS,Mississippi,NVSS,Alcohol,Chronic liver disease mortality,NaN,NaN,Number,...,OVERALL,OVR,NaN,NaN,NaN,NaN,2844658,32.56642,-89.593164,"{""type"":""Point"", ""coordinates"":[-89.593164,32...."
